# Power Flow via PandaModels
### PandaModels.jl: Interfacing PowerModels with pandapower

This tutorial describes how to run the Power Flow via [PandaModels.jl](https://e2niee.github.io/PandaModels.jl/dev/) calling [PowerModels.jl](https://lanl-ansi.github.io/PowerModels.jl/stable/) package.
Power Flow function in PowerModels is built based on [JuMP.jl](https://github.com/jump-dev/JuMP.jl) environment, thus same as OPF we need to set model parameter while calling PF function:

* "ACPPowerModel": a non-convex nonlinear AC power flow using complex voltages in polar coordinates
* "ACRPowerModel": a non-convex nonlinear AC power flow using complex voltages in rectangular coordinates
* "SOCWRPowerModel": a convex quadratic relaxation of the power flow problem
* "DCPPowerModel": a linear DC approximation of the power flow problem
* "DCMPPowerModel": a linear DC power flow model with polar voltage variables

Note: In order to compare the results of DC-PF between PowerModels and PYPOWER / matpower calculations,
 you need to set the model to "DCMPPowerModel". This direct current model should be equal to the results of matpower calculations,
  while model="DCPPowerModel" is the linearized model from AC model. For more information please check [here](https://lanl-ansi.github.io/PowerModels.jl/stable/formulation-details/#PowerModels.DCMPPowerModel).

### Let's get started

So here is an example of how it works. First, we create a simple grid in pandapower:

In [3]:
import copy
import numpy as np
import pandapower as pp
import pandapower.networks as nw

net = nw.simple_four_bus_system()
net.trafo.loc[0, "shift_degree"] = 0.

Then run ac power flow and get voltage angle and amplitude for buses:

In [5]:
try:
    pp.runpm_pf(net, pm_model="ACPPowerModel", calculate_voltage_angles=True)
except Exception as err:
    print(err)

hp.pandapower.opf.make_objective - WARNING: no costs are given - overall generated power is minimized


Cannot be performed


Also, there more parameters and options that you can add as input while calling the Optimization Problem from PandaModles:

| parameter | description | type | default |
| :--- | :--- | :---: | :--- |
| correct_pm_network_data | checks if network data is correct. If not tries to correct it | bool | True |
| silence | Suppresses information and warning messages output by PowerModels | bool | True |
| pm_model | PowerModels.jl model to use | str | "ACPPowerModel" |
| pm_solver | "main" solver| str | "ipopt" |
| pm_mip_solver | mixed integer solver| str | "cbc" |
| pm_nl_solver | nonlinear solver| str | "ipopt" |
| pm_tol | default desired convergence tolerance for solver to use | float | 1e-8 |
| pm_log_level | solver log level in power models | int | 0 |
| delete_buffer_file | If True, the .json file used by PandaModels will be deleted after optimization. | bool | True


In [6]:
va_pm = copy.deepcopy(net.res_bus.va_degree)
vm_pm = copy.deepcopy(net.res_bus.vm_pu)

run the powerflow from pypower:

In [7]:
pp.runpp(net)

In [8]:
va_pp = copy.deepcopy(net.res_bus.va_degree)
vm_pp = copy.deepcopy(net.res_bus.vm_pu)

lets compare the results:

In [9]:
np.allclose(va_pm, va_pp)

False

In [10]:
np.allclose(vm_pm, vm_pp)

False